# 初始化环境

In [ ]:
# 挂载谷歌云盘
import glob
from google.colab import drive
drive.mount('/Data')

# 设置环境变量函数
import os
def set_env(key: str, value) -> None:
  os.environ[key] = str(value)
def get_env(key: str) -> str:
  return os.environ.get(key, '')

# 配置环境
set_env('sovits', '/Data/My Drive/so-vits-svc-fork')
%cd "/Data/My Drive/so-vits-svc-fork"

# 安装/更新 Pytorch+SoVits
!python -m pip install -U pip setuptools wheel
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -U so-vits-svc-fork

# 预处理数据集(防手滑, 自行去除注释使用)

In [ ]:
# 防手贱
!sleep 100s
"""
# 预处理-音频重采样
!svc pre-resample
# 预处理-配置
!svc pre-config
# 预处理-生成Hubert与f0
!svc pre-hubert
"""

# 训练&自动打包AI模型

In [ ]:
# 添加执行权限
!chmod +0755 "${sovits}/train.sh"
# 执行自动训练&打包脚本
!"${sovits}/train.sh"

# 训练&自动打包AI模型(老版本, 无事务恢复)

In [ ]:
# 添加执行权限
!chmod +0755 "${sovits}/pack.sh"
# 执行打包脚本 并 训练模型
!"${sovits}/pack.sh" & svc train

# 多账号打包(已弃用)

In [ ]:
"""
# 计算模型版本
target_version = 0
for file in os.listdir(get_env('sovits') + '/logs/44k/'):
version_str = "".join(list(filter(str.isdigit, file)))
if len(version_str) == 0:
    continue
next_version = math.ceil(int(version_str) / 500) * 500 # 满整500备份
if next_version > target_version:
    target_version = next_version
set_env('target_version', target_version)
set_env('d_file', '{}/logs/44k/D_{}.pth'.format(get_env('sovits'), get_env('target_version')))
set_env('g_file', '{}/logs/44k/G_{}.pth'.format(get_env('sovits'), get_env('target_version')))
set_env('k_file', '{}/logs/44k/kmeans.pt'.format(get_env('sovits'), get_env('target_version')))

# 等待模型更新
!echo -n "目标版本: ${target_version} Epoch, 等待模型更新"
!until [[ -f "${d_file}" && -f "${g_file}" ]]; do echo -n "."; sleep 10s; done;
!echo ""
!sleep 5s

# 创建目录
set_env('output_dir', '{}/history_{}'.format(get_env('sovits'), get_env('target_version')))
!mkdir -p "${output_dir}"

# 训练-cluster模型
!svc train-cluster
# 移动模型
!mv "${k_file}" "${output_dir}/kmeans_${target_version}.pt"

# 拷贝模型
!cp "${d_file}" "${output_dir}/"
!cp "${g_file}" "${output_dir}/"

!if [ -f "${sovits}/history_${target_version}.7z" ] rm -rf "${sovits}/history_${target_version}.7z";
# 压缩保存
!7za a -t7z -m0=lzma -mx=9 -mfb=64 -md=32m -ms=on "${sovits}/history_${target_version}.7z" "${output_dir}" && rm -rf "${output_dir}";
"""

# 清理文件(防手滑, 自行去掉注释使用)

In [ ]:
# 清理(慎用)
# !svc clean